#### 지표 자동 UPDATE 
   - 주요 주식시장 지수 : spx, dji, ixix, kospi, kosdaq
   - 반도체지수, VIX 지수: sox, vix
   - 미국 채권 지수, 한국 채권 지수
   - 원화 환율
   - 선물 : 미국 3대 지수, wti, 달러지수 선물

In [1]:
# from bs4 import BeautifulSoup as bs
import requests
import datetime, time

import pandas as pd
import numpy as np

import os
import shutil

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [3]:
df_org = pd.read_csv('data/{}'.format('data_common.csv'))
df_org['date'] = df_org['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
df_org.set_index('date', inplace=True)
df_org.replace(np.nan, '', inplace=True) 

1	date
2	ixic
3	ixic_cr
4	ixic_f
5	ixic_f_cr
6	dji
7	dji_cr
8	dji_f
9	dji_f_cr
10	spx
11	spx_cr
12	spx_f
13	spx_f_cr
14	kospi
15	kospi_cr
16	kosdaq
17	kosdaq_cr
18	wti
19	wti_cr
20	dxy
21	dxy_cr
22	krw
23	krw_cr
24	bond_usa_10
25	bond_usa_10_cr
26	bond_usa_2
27	bond_usa_2_cr
28	bond_usa_3m
29	bond_usa_3m_cr
30	bond_kor_10
31	bond_kor_10_cr
32	bond_kor_2
33	bond_kor_2_cr
34	vix
35	vix_cr
36	f_g_index
37	sox
38	sox_cr
39	cpi
40	cpi_anticipated
41	cpi_previous
42	gold
43	gold_cr
44	fed_rate
45	fed_rate_fore
46	fed_rate_prev
47	fed_rate_ann
48	fed_rate_imp
49	bok_rate
50	bok_rate_fore
51	bok_rate_prev
52	bok_rate_ann
53	bok_rate_imp
54	fu_usa_date
55	op_usa_date
56	qw_usa_day
57	fu_kor_date
58	op_kor_date
59	dw_kor_day


In [4]:
spx_url = 'https://www.investing.com/indices/us-spx-500-historical-data'
dji_url = 'https://www.investing.com/indices/us-30-historical-data'
nas_url = 'https://www.investing.com/indices/nasdaq-composite-historical-data'
kospi_url = 'https://www.investing.com/indices/kospi-historical-data'
kosdaq_url = 'https://www.investing.com/indices/kosdaq-historical-data'
sox_url = 'https://www.investing.com/indices/phlx-semiconductor-historical-data'
# cpi_url = 'https://www.investing.com/economic-calendar/cpi-733'
vix_url = 'https://www.investing.com/indices/volatility-s-p-500-historical-data'

urls_major_index = [spx_url, dji_url, nas_url, kospi_url, kosdaq_url, sox_url, vix_url]


spx = ['date', 'spx', 'open', 'high', 'low', 'volume', 'spx_cr']
dji = ['date', 'dji', 'open', 'high', 'low', 'volume', 'dji_cr']
ixix = ['date', 'ixic', 'open', 'high', 'low', 'volume', 'ixic_cr']
kospi = ['date', 'kospi', 'open', 'high', 'low', 'volume', 'kospi_cr']
kosdaq = ['date', 'kosdaq', 'open', 'high', 'low', 'volume', 'kosdaq_cr']
sox = ['date', 'sox', 'open', 'high', 'low', 'volume', 'sox_cr']
# cpi = ['date', 'time', 'cpi', 'cpi_anticipated', 'cpi_previous', 'none']
# fng = ['date', 'n1', 'n2', 'n3', 'f_g_index']
vix = ['date', 'vix', 'open', 'high', 'low', 'volume', 'vix_cr']

col_names_major_index = [spx, dji, ixix, kospi, kosdaq, sox, vix ]

spx_s = ['date', 'spx', 'spx_cr']
dji_s = ['date', 'dji', 'dji_cr']
ixic_s = ['date', 'ixic', 'ixic_cr']
kospi_s = ['date', 'kospi', 'kospi_cr']
kosdaq_s = ['date', 'kosdaq', 'kosdaq_cr']
sox_s = ['date', 'sox', 'sox_cr']
# cpi_s = ['date', 'cpi', 'cpi_anticipated', 'cpi_previous']
# fng_s = ['date', 'f_g_index']
vix_s = ['date', 'vix', 'vix_cr']

col_names_s_major_index = [spx_s, dji_s, ixic_s, kospi_s, kosdaq_s, sox_s, vix_s]

In [5]:
us_bond_10yr_url = 'https://www.investing.com/rates-bonds/u.s.-10-year-bond-yield-historical-data'
us_bond_2yr_url = 'https://www.investing.com/rates-bonds/u.s.-2-year-bond-yield-historical-data'
us_bond_3mon_url = 'https://www.investing.com/rates-bonds/u.s.-3-month-bond-yield-historical-data'
kor_bond_10yr_url = 'https://www.investing.com/rates-bonds/south-korea-10-year-bond-yield-historical-data'
kor_bond_2yr_url = 'https://www.investing.com/rates-bonds/south-korea-2-year-bond-yield-historical-data'

urls_bonds = [us_bond_10yr_url, us_bond_2yr_url, us_bond_3mon_url,
        kor_bond_10yr_url, kor_bond_2yr_url]

# urls = [us_bond_10yr_url]

In [6]:
us_10yr = ['date', 'bond_usa_10', 'open', 'high', 'low', 'bond_usa_10_cr']
us_2yr = ['date', 'bond_usa_2', 'open', 'high', 'low', 'bond_usa_2_cr']
us_3mon = ['date', 'bond_usa_3m', 'open', 'high', 'low', 'bond_usa_3m_cr']
kor_10yr = ['date', 'bond_kor_10', 'open', 'high', 'low', 'bond_kor_10_cr']
kor_2yr = ['date', 'bond_kor_2', 'open', 'high', 'low','bond_kor_2_cr']

col_names_bonds = [us_10yr, us_2yr, us_3mon, kor_10yr, kor_2yr]

In [7]:
us_10yr_s = ['date', 'bond_usa_10', 'bond_usa_10_cr']
us_2yr_s = ['date', 'bond_usa_2', 'bond_usa_2_cr']
us_3mon_s = ['date', 'bond_usa_3m', 'bond_usa_3m_cr']
kor_10yr_s = ['date', 'bond_kor_10', 'bond_kor_10_cr']
kor_2yr_s = ['date', 'bond_kor_2', 'bond_kor_2_cr']

col_names_s_bonds = [us_10yr_s, us_2yr_s, us_3mon_s, kor_10yr_s, kor_2yr_s]

In [8]:
krw_rate_url = 'https://www.investing.com/currencies/usd-krw-historical-data'

urls_currencies = [krw_rate_url]

In [9]:
krw_rate = ['date', 'krw', 'open', 'high', 'low', 'vol', 'krw_cr'] ## 수정

col_names_currencies = [krw_rate]

In [10]:
krw_rate_s = ['date', 'krw', 'krw_cr']

col_names_s_currencies = [krw_rate_s]

In [11]:
nas_futures_url = 'https://www.investing.com/indices/nq-100-futures-historical-data'
snp_futures_url = 'https://www.investing.com/indices/us-spx-500-futures-historical-data'
dow_futures_url = 'https://www.investing.com/indices/us-30-futures-historical-data'
wti_futures_url = 'https://www.investing.com/commodities/crude-oil-historical-data'
dollar_index_url = 'https://www.investing.com/currencies/us-dollar-index-historical-data'

urls_future = [snp_futures_url, dow_futures_url,  nas_futures_url, wti_futures_url, dollar_index_url ]

snp_future = ['date', 'spx_f', 'open', 'high', 'low', 'volume', 'spx_f_cr']
dji_future = ['date', 'dji_f', 'open', 'high', 'low', 'volume', 'dji_f_cr']
ixic_future = ['date', 'ixic_f', 'open', 'high', 'low', 'volume', 'ixic_f_cr']
wti_future = ['date', 'wti', 'open', 'high', 'low', 'volume', 'wti_cr']
dxy_future = ['date', 'dxy', 'open', 'high', 'low', 'volume', 'dxy_cr']

col_names_future = [snp_future,dji_future,ixic_future, wti_future, dxy_future]

snp_future_s = ['date', 'spx_f', 'spx_f_cr']
dji_future_s = ['date', 'dji_f', 'dji_f_cr']
ixic_future_s = ['date', 'ixic_f', 'ixic_f_cr']
wti_future_s = ['date', 'wti', 'wti_cr']
dxy_future_s = ['date', 'dxy', 'dxy_cr']

col_names_s_future  = [snp_future_s, dji_future_s, ixic_future_s, wti_future_s, dxy_future_s]


In [12]:
urls = urls_major_index + urls_bonds + urls_currencies + urls_future
col_names = col_names_major_index + col_names_bonds + col_names_currencies + col_names_future
col_names_s = col_names_s_major_index + col_names_s_bonds + col_names_s_currencies + col_names_s_future

In [13]:
# urls = urls[2:3]
# col_names = col_names[2:3]
# col_names_s = col_names_s[2:3]

In [14]:
def make_datetime_series_df(df, start_date, end_date):
    # make dataframe with no empty date index with input df.
    # start_date =  '2021-01-01' str type.
    # df.index = datetime series
    # df.columns = ['a', 'b',,,,]
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_date = pd.DataFrame(columns = df.columns)
    df_date.insert(0, 'date', date_range_ts)
    df_date.replace(np.nan, '', inplace=True) # Nan을 빈 칸으로 대체
    df_date.set_index('date', inplace=True)
    df_date.update(df)
    return df_date

In [15]:
# url = 'https://kr.investing.com/indices/us-spx-500-historical-data'
# res = requests.get(url, headers=headers)
# df = pd.read_html(res.text, attrs={"data-test": "historical-data-table"}, flavor=["lxml", "bs4"])[0]

In [16]:
for i, url in enumerate(urls):
    print('url', url)
    # 시간, 미국, 한국 접속사이트에 따라 attribute 가 변경되서 나타나기 때문에 error  처리를 위해 try 사용
    count = 0
    while True:
        try :
            res = requests.get(url, headers=headers)
            df = pd.read_html(res.text, attrs={"id": "curr_table"}, flavor=["lxml", "bs4"])[0]
            break
        except:
            res = requests.get(url, headers=headers)
            df = pd.read_html(res.text, attrs={"data-test": "historical-data-table"}, flavor=["lxml", "bs4"])[0]
            break
        finally:
            time.sleep(1)
            count += 1
            if count > 5 :
                raise ValueError('The url request is delaying')
#                 break
                
    print(url)
    print(i, col_names[i])
    df.columns = col_names[i]
    
    # 시간, 미국, 한국 접속사이트에 따라 attribute 가 변경되서 나타나기 때문에 error  처리를 위해 try 사용
    try:
        df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%b %d, %Y"))
    except:
        df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%m/%d/%Y"))
    
    df.sort_values(by=['date'], inplace=True)
    df.drop_duplicates(subset=['date'], inplace=True)
    df_select = df[col_names_s[i]]
    df_select.set_index('date', inplace=True)
    
    start_date = df_select.index[0]
    end_date = df_select.index[-1]
    df_select = make_datetime_series_df(df_select, start_date, end_date)

    df_org.update(df_select)

url https://www.investing.com/indices/us-spx-500-historical-data


ValueError: No tables found

#### get append cpi

In [ ]:
cpi_url = 'https://www.investing.com/economic-calendar/cpi-733'

In [ ]:
res = requests.get(cpi_url, headers=headers)
df = pd.read_html(res.text, flavor=["lxml", "bs4"])[0]
df.columns = ['date', 'time', 'cpi', 'cpi_anticipated', 'cpi_previous', 'none']
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:12], "%b %d, %Y"))

df.sort_values(by=['date'], inplace=True)
df.drop_duplicates(subset=['date'], inplace=True)
df_select = df[['date', 'cpi', 'cpi_anticipated', 'cpi_previous']]
df_select.set_index('date', inplace=True)

start_date = df_select.index[0]
end_date = df_select.index[-1]
df_select = make_datetime_series_df(df_select, start_date, end_date)

df_org.update(df_select)

#### get append fear and greed

In [ ]:
import pytz, json

In [ ]:
today = datetime.date.today()
today_p = today.strftime('%Y%m%d')
diff_days = datetime.timedelta(days=30)
today = today - diff_days
start_date = today.strftime('%Y-%m-%d')

In [ ]:
BASE_URL = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
# START_DATE = '2022-01-01'
START_DATE = start_date

In [ ]:
def transform_timezone(x):
    dt = datetime.datetime.fromtimestamp(x / 1000, tz=pytz.utc)
#     tzone = pytz.timezone('US/Eastern')
    tzone = pytz.timezone('Asia/Seoul')
    loc_dt = dt.astimezone(tzone)
    return loc_dt.date()

In [ ]:
r = requests.get("{}/{}".format(BASE_URL, START_DATE), headers=headers)
data = r.json()
fg_data = data['fear_and_greed_historical']['data']

In [ ]:
df = pd.DataFrame(fg_data)
df.columns = ['date', 'f_g_index', 'rating']
# df.head()

In [ ]:
df['date'] = df['date'].apply(lambda x: transform_timezone(x))
df['f_g_index'] = df['f_g_index'].astype('int')

df.sort_values(by=['date'], inplace=True)
df.drop_duplicates(subset=['date'], inplace=True) # 중복항목 삭제. 필수 검토해야 함. 중복입력되는 경우 다수 발생

df_select = df[['date', 'f_g_index']]
df_select.set_index('date', inplace=True)

start_date = df_select.index[0]
end_date = df_select.index[-1]
df_select = make_datetime_series_df(df_select, start_date, end_date)

df_org.update(df_select)

In [ ]:
# df_org.reset_index(inplace=True)
# df_org.to_csv('data/data_common.csv', index=False)

#### get and append gold price

In [ ]:
gold_url = 'https://www.usagold.com/daily-gold-price-history/'

In [ ]:
res = requests.get(gold_url, headers=headers)
df = pd.read_html(res.text, flavor=["lxml", "bs4"])
df = df[0].drop(0) # delete empty first row
df.columns = ['date', 'gold']
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%d %b %Y"))
df.sort_values(by=['date'], inplace=True)
df.drop_duplicates(subset=['date'], inplace=True) 

In [ ]:
# calculate change_rate
df_shift_p1 = df['gold'].shift(1)
df['gold_cr'] = (df['gold'] - df_shift_p1)/df_shift_p1
df.replace(np.nan, '', inplace=True)
# df.head()

In [ ]:
df.set_index('date', inplace=True)

start_date = df.index[0]
end_date = df.index[-1]
df = make_datetime_series_df(df, start_date, end_date)

df_org.update(df)

### fed 금리 get append

In [ ]:
interest_url = 'https://www.investing.com/economic-calendar/interest-rate-decision-168/'

In [ ]:
res = requests.get(interest_url, headers=headers)
df = pd.read_html(res.text, flavor=["lxml", "bs4"])[0]
# df.head()

In [ ]:
df.columns = ['date', 'time', 'fed_rate', 'fed_rate_fore', 'fed_rate_prev', 'none']

In [ ]:
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:12], "%b %d, %Y"))
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df.sort_values(by=['date'], inplace=True)
df.drop_duplicates(subset=['date'], inplace=True) 

In [ ]:
df_select = df[['date', 'fed_rate', 'fed_rate_fore', 'fed_rate_prev']]
df_select.set_index('date', inplace=True)

start_date = df_select.index[0]
end_date = df_select.index[-1]
df_select = make_datetime_series_df(df_select, start_date, end_date)

df_org.update(df_select)

### 한국은행 금리 get append

In [ ]:
kor_url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'

In [ ]:
res = requests.get(kor_url, headers=headers)
df = pd.read_html(res.text, attrs = {'class': 'fixed'}, flavor=["lxml", "bs4"])[0]

In [ ]:
df.columns=  ['release_yr', 'release_date', 'bok_rate']
df_date_temp = df['release_yr'].astype('str')+df['release_date']
df['date'] = df_date_temp.apply(lambda x : datetime.datetime.strptime(x, "%Y%m월 %d일"))
df.sort_values(by=['date'], inplace=True)
df.drop_duplicates(subset=['date'], inplace=True) 

In [ ]:
df_select = df[['date', 'bok_rate']] # leave only valid columns
df_select.set_index('date', inplace=True)

df_org.update(df_select)

### make csv file

In [ ]:
df_org.reset_index(inplace=True)
df_org.to_csv('data/data_common.csv', index=False)

In [ ]:
df_org.set_index('date').loc['2022-08-01':'2022-08-12', ['ixic', 'dji', 'spx']]